In [12]:
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
from sklearn.impute import KNNImputer

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_csv("../working_data/dataset.csv", index_col=0)

In [14]:
df = df[df["Category"] != "Industrial Crops"]

## Creating working data frame

In [15]:
#df['Year'] = pd.to_datetime(df['Year'], format='%Y')
#df.set_index("Year", inplace=True)
# getting list of names
categories = df["Category"].unique()

## Creating the metrics data frame

In [16]:
def calculate_metrics(df, crop_name, y_true, y_pred):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    y_test_avg = pd.DataFrame(y_true).mean()
    y_pred_avg = y_pred.mean()

    df.loc[crop_name] = [mae, mse, rmse, r2, y_test_avg[0], y_pred_avg]

In [17]:
quality_metrics = ['xgb_MAE', 'xgb_MSE', 'xgb_RMSE', "xgb_R2", "Actual Yield", "Predicted Yield"]
error_df = pd.DataFrame(columns = quality_metrics, index = categories)
#display(error_df)

## Gradient Boosting Regressor

### Setting up dataframe

#### - Solving for stationarity

In [18]:
imputer = KNNImputer(n_neighbors=4)
adf_df = pd.DataFrame(columns=df.columns)

for cate in categories: 
    # Selects category
    crop_df = df[df["Category"] == cate]
    num_cols = crop_df.select_dtypes(include=[np.number])
    # Selects column for ADF Test
    for col in num_cols:
        if crop_df[col].count() < 5:
            continue
        adf_result = adfuller(crop_df[col])
        while adf_result[1] > 0.05:
            crop_df[col] = crop_df[col].diff()
            crop_df[num_cols.columns] = imputer.fit_transform(crop_df[num_cols.columns])
            adf_result = adfuller(crop_df[col])

    adf_df = pd.concat([adf_df, crop_df])

#### - Transforming time series data to supervised learning (adding lags to variables)

In [19]:
main_df = pd.DataFrame(columns=adf_df.columns)
imputer = KNNImputer(n_neighbors=4)

# Sliding Window
for cate in categories: 
    # Selects category
    crop_df = adf_df[adf_df["Category"] == cate]
    num_cols = crop_df.select_dtypes(include=[np.number])
    for col in num_cols.columns:   
        for i in range(1, 3):
            crop_df[f'{col}-{i}'] = crop_df[col].shift(i)

    main_df = pd.concat([main_df, crop_df])
    num_cols = main_df.select_dtypes(include=[np.number])
    main_df[num_cols.columns] = imputer.fit_transform(main_df[num_cols.columns])

## Running the model

In [27]:
scaler = MinMaxScaler()
for cat in categories:
   crop_df = main_df[main_df['Category'] == cat]
   encoded_df = pd.get_dummies(crop_df[["Season"]], prefix='', prefix_sep='')
   encoded_df = encoded_df.drop(encoded_df.columns[-1], axis=1)

   y = crop_df[['Yield (tonnes/ha)']]
   x = crop_df.drop(['Yield (tonnes/ha)', "Item", "Category", "Season"], axis=1)
   x = pd.concat([x, encoded_df], axis=1)
   

   mean = x.mean()
   std = x.std()
   lower_bound = mean - 2 * std
   upper_bound = mean + 2 * std
   x = x.clip(lower_bound, upper_bound, axis=1)

   x = scaler.fit_transform(x)

   x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
   regressor = GradientBoostingRegressor(n_estimators=1000, random_state=42, learning_rate=0.2)
   regressor.fit(x_train, y_train)

   #prediction of new value
   y_pred = regressor.predict(x_test)
   calculate_metrics(error_df, cat, y_test, y_pred)

## Result

In [28]:
error_df

,xgb_MAE,xgb_MSE,xgb_RMSE,xgb_R2,Actual Yield,Predicted Yield
Vegetables and Legumes,17.593632,934.026008,30.561839,0.71929,45.004377,39.283707
Cereals,0.681111,0.758598,0.870975,0.881291,0.022847,0.164165
Fruits and Nuts,4.360747,34.982691,5.914617,0.358406,9.75563,11.156794
